In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/Autumn_2025/42028_Deep_Learning/Assignments/assignment 3/code

/content/drive/MyDrive/Autumn_2025/42028_Deep_Learning/Assignments/assignment 3/code


In [3]:
ls

 app.log             face_models/          runs_bad2/        yolo11n.pt
 bytrack/            facial_recognition/   runs_bad3/       'yolo12m (1).pt'
'data collection'/   human_face/           runs_old_goog/    yolo12m.pt
 datasets/          'Prevous yolo runs'/   sounds/           yolo12n.pt
 detect/             README.md             ui/               yolo12s.pt
 drone/              runs/                 yolo/            'yolo models'/
 face_data/          runs_bad/            'yolo11n (1).pt'


In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

  from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Wed May 14 12:34:47 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   53C    P8             14W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# ⚙️ 1. Setup paths and parameters


In [5]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 129.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 102.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 110.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [6]:
import torch
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("GPU memory cleared.")
else:
    print("No GPU available.")

GPU memory cleared.


In [7]:
import os
import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [8]:
database = "/content/drive/MyDrive/Autumn_2025/42028_Deep_Learning/Assignments/assignment 3/code/datasets/mix dataset"


In [9]:
data_yaml_path = f"{database}/data.yaml"


In [10]:
data_yaml_path

'/content/drive/MyDrive/Autumn_2025/42028_Deep_Learning/Assignments/assignment 3/code/datasets/mix dataset/data.yaml'

In [11]:
# Load a model
model = YOLO("yolo12s.yaml")  # build a new model from YAML
model = YOLO("yolo12s.pt")  # load a pretrained model (recommended for training)
model = YOLO("yolo12s.yaml").load("yolo12s.pt")  # build from YAML and transfer weights

Transferred 691/691 items from pretrained weights


In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
# Train the model
results = model.train(
    data=data_yaml_path,
    epochs=500,
    imgsz=640,
    batch=64,
    workers=8,
    pretrained=True,
    cache='ram',
    device= device,
    optimizer='AdamW',
    lr0=0.001,
    cos_lr=True,
    warmup_epochs=3,
    lrf=0.01,
    weight_decay=0.0005,
    momentum=0.937,
    project='runs/yolov12s',
    name='train',
    exist_ok=True,
    save_period = 50
)



engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=ram, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/drive/MyDrive/Autumn_2025/42028_Deep_Learning/Assignments/assignment 3/code/datasets/mix dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=500, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12s.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretr

100%|██████████| 755k/755k [00:00<00:00, 19.4MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  3                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  4                  -1  1    103360  ultralytics.nn.modules.block.C3k2            [128, 256, 1, False, 0.25]    
  5                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  6                  -1  2    689408  ultralytics.nn.modules.block.A2C2f           [256, 256, 2, True, 4]        
  7                  -1  1   1180672  ultralytics

 20                  -1  1   1511424  ultralytics.nn.modules.block.C3k2            [768, 512, 1, True]           
 21        [14, 17, 20]  1    819795  ultralytics.nn.modules.head.Detect           [1, [128, 256, 512]]          
YOLO12s summary: 272 layers, 9,253,523 parameters, 9,253,507 gradients, 21.5 GFLOPs

Transferred 685/691 items from pretrained weights
Freezing layer 'model.21.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.5±0.1 ms, read: 0.1±0.1 MB/s, size: 61.7 KB)


train: Scanning /content/drive/MyDrive/Autumn_2025/42028_Deep_Learning/Assignments/assignment 3/code/datasets/mix dataset/train/labels.cache... 3228 images, 6 backgrounds, 0 corrupt: 100%|██████████| 3234/3234 [00:00<?, ?it/s]


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    461/500      21.7G     0.4602     0.3193     0.9537         74        640: 100%|██████████| 51/51 [00:59<00:00,  1.17s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.28it/s]

                   all        165        186      0.944      0.887      0.947      0.792



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    462/500      21.7G     0.4389     0.2997     0.9451        124        640:  24%|██▎       | 12/51 [00:14<00:46,  1.18s/it]

# 7. 📊Visualization the results

In [ ]:
# Load your trained model
model = YOLO("runs/yolov12s/train/weights/best.pt")

# Evaluate on the validation set defined in your data.yaml
metrics = model.val()




In [ ]:
# Mean metrics across all classes
print(f"mAP@0.5: {metrics.box.map50:.3f}")
print(f"mAP@0.5:0.95: {metrics.box.map:.3f}")
print(f"Mean Precision: {metrics.box.mp:.3f}")
print(f"Mean Recall: {metrics.box.mr:.3f}")



In [ ]:
p, r, ap50, ap = metrics.box.class_result(0)
print(f"Human Class — Precision: {p:.3f}, Recall: {r:.3f}, AP@0.5: {ap50:.3f}, AP@0.5:0.95: {ap:.3f}")


# continue training later from the last checkpoint

In [ ]:
# # Load a model
# model = YOLO("runs/yolov12m-human/train/weights/epoch.pt")  # load a partially trained model

# # Resume training
# results = model.train(
#     epochs=4,
#     imgsz=640,
#     batch=32,
#     workers=8,
#     pretrained=True,
#     cache='ram',
#     device='cuda:0',
#     optimizer='AdamW',
#     lr0=0.001,
#     cos_lr=True,
#     warmup_epochs=3,
#     lrf=0.01,
#     weight_decay=0.0005,
#     momentum=0.937,
#     project='runs/yolov12m-human',
#     name='continue train',
#     exist_ok=True,
#     save_period = 1,
#     resume=True
# )

In [ ]:

# # Load your trained model
# model = YOLO("runs/yolov12m-human/train/weights/epoch3.pt")

# # Evaluate on the validation set defined in your data.yaml
# metrics = model.val()




In [ ]:
# # Mean metrics across all classes
# print(f"mAP@0.5: {metrics.box.map50:.3f}")
# print(f"mAP@0.5:0.95: {metrics.box.map:.3f}")
# print(f"Mean Precision: {metrics.box.mp:.3f}")
# print(f"Mean Recall: {metrics.box.mr:.3f}")



In [ ]:
#p, r, ap50, ap = metrics.box.class_result(0)
#print(f"{name} — Precision: {p:.3f}, Recall: {r:.3f}, AP@0.5: {ap50:.3f}, AP@0.5:0.95: {ap:.3f}")
